In [1]:
import torch, os
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torch.utils.data as data
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights
from torch.nn import functional as F

from pytorch_lightning import LightningModule, Trainer
import torchmetrics


In [2]:
# import neptune.new as neptune
from pytorch_lightning.loggers import NeptuneLogger

# run = neptune.init_run(
#     project="kaori/XAI",  
#     api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIyZjZiMDA2YS02MDM3LTQxZjQtOTE4YS1jODZkMTJjNGJlMDYifQ==",
# )

neptune_logger = NeptuneLogger(
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIyZjZiMDA2YS02MDM3LTQxZjQtOTE4YS1jODZkMTJjNGJlMDYifQ==",
    project="kaori/XAI",
    # tags=["simple", "showcase"],
    log_model_checkpoints=False,
)

params = {
    #dataset
    "dataset": "Caltech256",
    "data_dir": "dataset/Caltech1-20/",
    "input_sz": (224, 224),
    "n_classes": 20,

    #model
    "model": "ResNet18",

    #Training
    "lr": 1e-4,
    "bs": 64,
    "max_epochs": 10,
    
    #checkpoint
    "ckpt_path": "1"
    
}
# run["parameters"] = params

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(20),
        transforms.RandomHorizontalFlip(0.5),
        transforms.Resize(params['input_sz']),
        transforms.ToTensor(),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(params['input_sz']),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
    ]),
    'test': transforms.Compose([
        transforms.Resize(params['input_sz']),
        transforms.ToTensor(),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
    ])
}

dataset = datasets.ImageFolder(params["data_dir"], transform=data_transforms["val"])
n = len(dataset)
train_set, val_set = data.random_split(dataset, [int(0.8 * n), n - int(0.8 * n)])

trainloader = DataLoader(train_set, batch_size=params["bs"], shuffle=True, num_workers=4)
valloader = DataLoader(val_set, batch_size=params["bs"], shuffle=False, num_workers=4)

In [4]:
class Classifer(LightningModule):
    def __init__(self, n_classes):
        super().__init__()
        self.model_ft = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        # self.model_ft.avgpool = nn.AdaptiveAvgPool2d(1)
        n_features = self.model_ft.fc.in_features
        self.model_ft.fc = nn.Linear(n_features, n_classes)

        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes)
        self.valid_acc = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes)
        

    def forward(self, x):
        # out = self.model_ft(x)
       
        return self.model_ft(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("batch/train_loss", loss, prog_bar=False)

        y_pred = y_hat.argmax(axis=1)
        self.train_acc(y_pred, y)
        self.log("batch/train_acc", self.train_acc, prog_bar=True)

        return {"loss": loss}

    def training_epoch_end(self, outputs):
        self.log('train_acc', self.train_acc.compute())
        self.train_acc.reset()
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_pred = y_hat.argmax(axis=1)
       
        self.valid_acc(y_pred, y)

    def validation_epoch_end(self, outputs):
        self.log('val_acc', self.valid_acc.compute())
        self.valid_acc.reset()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=params["lr"])

In [5]:
from pytorch_lightning.callbacks import ModelCheckpoint
torch.set_float32_matmul_precision('medium')

model = Classifer(params["n_classes"])

# (neptune) log hyper-parameters
neptune_logger.log_hyperparams(params=params)

checkpoint_callback = ModelCheckpoint(dirpath=os.path.join("model", params["ckpt_path"]), save_top_k=2, monitor="val_acc", mode="max")

# (neptune) initialize a trainer and pass neptune_logger
trainer = Trainer(
    logger=neptune_logger,
    max_epochs=params["max_epochs"],
    accelerator="gpu",
    devices=1,
    callbacks=[checkpoint_callback]
)



# train the model log metadata to the Neptune run
trainer.fit(model=model, train_dataloaders=trainloader, val_dataloaders=valloader)
neptune_logger.finalize(status=)

https://app.neptune.ai/kaori/XAI/e/XAI-41
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


c:\Users\197796\Anaconda3\envs\torch\lib\site-packages\neptune\common\deprecation.py:45: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` or `repr(object)` instead. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | model_ft  | ResNet             | 11.2 M
1 | train_acc | MulticlassAccuracy | 0     
2 | valid_acc | MulticlassAccuracy | 0     
-------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.747    Total estimated model

Sanity Checking: 0it [00:00, ?it/s]

c:\Users\197796\Anaconda3\envs\torch\lib\site-packages\pytorch_lightning\trainer\trainer.py:1609: PossibleUserWarning: The number of training batches (32) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


TypeError: finalize() missing 1 required positional argument: 'status'